In [25]:
# Використати техніки об’єднання, злиття, транспонування і приєднання додаткових даних.
import pandas as pd
import numpy as np

diabetes_data = pd.read_csv("../data/diabetes_non_nulled.csv")


In [26]:
# ------ join ------

# Я вирішив додати якісь загальні значення відповідні до метрик з нашого датасету.
# Для цього додав табличку по BMI
bmi_ref = pd.read_csv('../data/bmi_ref.csv')
bmi_ref

,bmi_cat,bmi_min,bmi_max,severity_rank,risk_note
0,underweight,-inf,18.5,0,low risk
1,normal,18.5,25.0,1,baseline risk
2,overweight,25.0,30.0,2,elevated cardiometabolic risk
3,obesity,30.0,inf,3,high cardiometabolic risk


In [33]:
# Оскільки у нас є лише значення BMI, а для категорій потрібні діапазони, то варто підрахувати
# і тим сами створити ключ для обʼєднання
def choose_category(bmi):
    for cat in bmi_ref.index:
        if bmi < bmi_ref.loc[cat, 'bmi_max'] and bmi >= bmi_ref.loc[cat, 'bmi_min']:
            return bmi_ref.loc[cat, 'bmi_cat']


diabetes_data['bmi_cat'] = diabetes_data['BMI'].apply(lambda x: choose_category(x))
diabetes_data.head(10)

# Тепер можемо обʼєднати і доповнити відповідними даними з таблички по BMI
diabetes_data_enriched = diabetes_data.merge(bmi_ref, on="bmi_cat", how="left")

# diabetes_data_enriched.groupby('risk_note')['Outcome'].sum()


In [66]:
# ------ pivot ------


# diabetes_data_enriched.pivot_table(
#     index="severity_rank",
#     columns="Outcome",
#     values="row_id",
#     aggfunc="count",
#     fill_value=0
# )

# У відсотках
(
    diabetes_data_enriched.pivot_table(
        index="severity_rank",
        columns="Outcome",
        values="row_id",
        aggfunc="count",
        fill_value=0
    )
    .pipe(lambda x: round(x.div(x.sum(axis=1), axis=0) * 100, 2))
)

# Як бачимо то маємо чітку відповідність між рівнем ризику для Індексу маси тіла та наявності діабету.
# Важко сказати чи це причина чи наслідок, оскільки діабет може спиричиняти набір ваги, проте кореляція досить чітка.

Outcome,0,1
severity_rank,,
0,100.00,0.00
1,93.14,6.86
2,77.65,22.35
3,54.24,45.76


In [69]:
# ------ unstack ------

metrics = ["Glucose", "BloodPressure", "Age"]

s = diabetes_data_enriched.groupby(["severity_rank", "Outcome"])[metrics].median()
wide = s.unstack("Outcome")
print(wide)

# Glucose - видно чітку різницю між Outcome 0/1 незалежно від severity_rank.
# BloodPressure - не так сильно, але теж підвищений для групи з Outcome=1, також зростає відповідно до підняття severity_rank
# Age - в групі Outcome=1 медіана віку більша для всіх severity_rank, але не корелює з severity_rank як таким


              Glucose        BloodPressure         Age      
Outcome             0      1             0     1     0     1
severity_rank                                               
0                97.0    NaN          69.0   NaN  24.0   NaN
1               101.0  162.0          66.0  72.0  26.0  50.0
2               109.0  124.5          68.0  73.0  27.0  36.0
3               108.0  142.0          72.0  76.0  27.0  36.0
